In [3]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras import models, layers
from keras.optimizers import Adam
from sklearn import metrics

In [4]:
train_folder = '/kaggle/input/splitted-data/UCMerced_LandUse_Splitted/Train'
all_data = []
for folder in os.listdir(train_folder):
    
    label_folder = os.path.join(train_folder, folder)
    onlyfiles = [{'label':folder,'path':os.path.join(label_folder, f)} for f in os.listdir(label_folder) if os.path.isfile(os.path.join(label_folder, f))]
    #print(onlyfiles)
    all_data += onlyfiles
data_df = pd.DataFrame(all_data)
data_df

,label,path
0,airplane,/kaggle/input/splitted-data/UCMerced_LandUse_S...
1,airplane,/kaggle/input/splitted-data/UCMerced_LandUse_S...
2,airplane,/kaggle/input/splitted-data/UCMerced_LandUse_S...
3,airplane,/kaggle/input/splitted-data/UCMerced_LandUse_S...
4,airplane,/kaggle/input/splitted-data/UCMerced_LandUse_S...
...,...,...
1693,agricultural,/kaggle/input/splitted-data/UCMerced_LandUse_S...
1694,agricultural,/kaggle/input/splitted-data/UCMerced_LandUse_S...
1695,agricultural,/kaggle/input/splitted-data/UCMerced_LandUse_S...
1696,agricultural,/kaggle/input/splitted-data/UCMerced_LandUse_S...


In [5]:
img_width, img_height = 256, 256
batch_size = 100
y_col = 'label'
x_col = 'path'
no_of_classes = len(data_df[y_col].unique())

In [7]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
  "/kaggle/input/splitted-data/UCMerced_LandUse_Splitted/Train",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical")

val_ds = tf.keras.utils.image_dataset_from_directory(
  "/kaggle/input/splitted-data/UCMerced_LandUse_Splitted/Test/",
  image_size=(img_height, img_width),
  batch_size=batch_size,
  label_mode="categorical")


Found 1698 files belonging to 21 classes.
Found 420 files belonging to 21 classes.


In [8]:
base_model = ResNet50(weights = "imagenet", include_top = False, input_shape = (256, 256, 3))
base_model.trainable = False ## Immutable base model

94765736/94765736 [==============================] - 1s 0us/step


In [9]:
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(256, activation='relu')
dropout_layer_1 = layers.Dropout(0.5)
prediction_layer = layers.Dense(21, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dropout_layer_1,
    prediction_layer
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 256)               33554688  
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 21)                5397      
                                                                 
Total params: 57,147,797
Trainable params: 33,560,085
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
classes = 21
epochs = 40
learning_rate = 0.0001

adam = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_ds,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_ds, shuffle=True)


Epoch 1/40
17/17 [==============================] - 26s 720ms/step - loss: 1.8248 - accuracy: 0.5501 - val_loss: 0.4204 - val_accuracy: 0.8452
Epoch 2/40
17/17 [==============================] - 9s 472ms/step - loss: 0.2558 - accuracy: 0.9164 - val_loss: 0.1929 - val_accuracy: 0.9333
Epoch 3/40
17/17 [==============================] - 9s 473ms/step - loss: 0.0886 - accuracy: 0.9717 - val_loss: 0.1852 - val_accuracy: 0.9476
Epoch 4/40
17/17 [==============================] - 9s 482ms/step - loss: 0.0699 - accuracy: 0.9776 - val_loss: 0.1464 - val_accuracy: 0.9429
Epoch 5/40
17/17 [==============================] - 9s 481ms/step - loss: 0.0502 - accuracy: 0.9847 - val_loss: 0.1905 - val_accuracy: 0.9333
Epoch 6/40
17/17 [==============================] - 9s 497ms/step - loss: 0.0312 - accuracy: 0.9882 - val_loss: 0.1816 - val_accuracy: 0.9357
Epoch 7/40
17/17 [==============================] - 9s 488ms/step - loss: 0.0277 - accuracy: 0.9912 - val_loss: 0.1743 - val_accuracy: 0.9500
Epoch

In [12]:
print("Training Accuracy: ", history.history['accuracy'][-1]*100, "%")

Training Accuracy:  99.41107034683228 %


In [13]:
print("Testing Accuracy: ", history.history['val_accuracy'][-1]*100, "%")

Testing Accuracy:  93.8095211982727 %
